In [1]:
# Creating a custom dataset for Pytorch

In [1]:
# Imports
#from PIL import Image
#from natsort import natsorted
import os
import rasterio
import imagesize
import numpy as np
from rasterio.windows import Window, bounds, from_bounds
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from dl_toolbox.augmentations import get_transforms

ModuleNotFoundError: No module named 'dl_toolbox'

In [2]:
import os
from itertools import product
import rasterio as rio
from rasterio import windows

def get_tiles(image_path, width=256, height=256):
    with rasterio.open(image_path) as ds : 
        print(type(ds))
        nols, nrows = ds.meta['width'], ds.meta['height']
        offsets = product(range(0, nols, width), range(0, nrows, height))
        big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
        for col_off, row_off in  offsets:
            window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
            yield window
            
with rio.open(os.path.join(in_path, input_filename)) as inds:
    tile_width, tile_height = 256, 256
    meta = inds.meta.copy()
    for window in get_tiles(inds):
        print(window)
        meta['transform'] = transform
        meta['width'], meta['height'] = window.width, window.height
        outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
        with rio.open(outpath, 'w', **meta) as outds:
            outds.write(inds.read(window=window))

In [3]:
# creating aclass called InriaDS, pass the argument Dataset 
# to allow it to inherit the functionality of the Torch Dataset Class

class InriaDS(Dataset):
    # definition of the init func to initialize ou variables
    def __init__(self, image_path, tile,fixed_crops, img_dim,img_aug = 'no',label_path= None, *args, **kwargs):         
        
        self.image_path = image_path # base path to InriaDS folder img
        self.label_path = label_path # base path to InriaDS folder gt
        #self.tif_name = tif_name     # img name
        self.tile = tile             # tile off of img and label
        self.crop_windows = list(get_tiles(image_path, tile.width, tile.height)) if fixed_crops else None
        self.img_dim = img_dim
        self.img_aug = get_transforms(img_aug)
        
    def __len__(self):
        return len(self.crop_windows) if self.crop.windows else 1
    
    def __getitem(self, idx):
        
        if self.crop_windows :
            window = self.crop_windows[idx]
        else :
            cx = self.tile.col_off + np.random.randint(0, self.tile.width - self.crop_size + 1)
            cy = self.tile.row_off + np.random.randint(0, self.tile.width - self.crop_size + 1)
            window = Window(cx,cy,self.img_dim,self.img_dim)
        
        with rasterio.open(self.image_path) as image_file:
            image = image_file.read(window= window, out_dtype = np.float32)
        image = torch.from_numpy(image).float().contiguous()
        
        label = None 
        if self.label_path:
            with rasterio.open(self.label_path) as label_file:
                label = label_file.read(window= window, out_dtype = np.float32)
            label = torch.from_numpy(label).float().contiguous()
            
        if self.img_aug is None :            
            end_image, end_mask = image, label
            
        return {'orig_image': image, 'orig_mask':label, 'image': end_image, 'window': window, 'mask':end_mask}

In [8]:
img_path = 'C:/Users/marie/ML/aerialimagelabeling/NEW2-AerialImageDataset/AerialImageDataset/train/images/austin11.tif'
lbl_path = 'C:/Users/marie/ML/aerialimagelabeling/NEW2-AerialImageDataset/AerialImageDataset/train/gt/austin11.tif'

def main():
    
    dataset = InriaDS(image_path = img_path, label_path = lbl_path, tile = Window(col_off=0, row_off=0, width=400, height=400), img_aug = 'no', img_dim = 256, fixed_crops = True)
    print(type(dataset))
    for data in dataset :
        print(data['window'])

In [9]:
if __name__ == '__main__':
    main()

<class 'rasterio.io.DatasetReader'>
<class '__main__.InriaDS'>


NotImplementedError: 